# Первая задача

In [ ]:
import json
import pandas as pd
import sqlite3

def parse_file(file_name):
    df = pd.read_pickle(file_name)
    return df


def connect_to_db(file_name):
    connection = sqlite3.connect(file_name)
    connection.row_factory = sqlite3.Row

    return connection


def insert_data(db, data):
    cursor = db.cursor()
    cursor.executemany(f"""
        INSERT INTO tournaments (id, name, city, begin, system, tours_count, min_rating, time_on_game)
        VALUES(
            :id, :name, :city, :begin, :system, :tours_count, :min_rating, :time_on_game
        )
    """, data)

    db.commit()


def get_top_by_rating(db, limit):
    cursor = db.cursor()
    res = cursor.execute("SELECT * FROM tournaments ORDER BY min_rating DESC LIMIT ?", [limit])
    items = list()

    for row in res.fetchall():
        items.append(dict(row))
    cursor.close()

    return items


def get_stat_by_time(db):
    cursor = db.cursor()
    res = cursor.execute("""
        SELECT
            SUM(time_on_game) as sum,
            AVG(time_on_game) as avg,
            MIN(time_on_game) as min,
            MAX(time_on_game) as max
        FROM tournaments
    """)

    result = dict(res.fetchone())
    cursor.close()

    return result


def get_freq_by_system(db):
    cursor = db.cursor()
    res = cursor.execute("""
        SELECT
            system,
            COUNT(id) as cnt
        FROM tournaments
        GROUP BY system
    """)

    result = list()

    for row in res.fetchall():
        result.append(dict(row))
    cursor.close()

    return result


def filter_by_tours(db, min_tours_count, limit=10):
    cursor = db.cursor()
    res = cursor.execute("""
        SELECT *
        FROM tournaments
        WHERE tours_count > ?
        ORDER BY min_rating DESC
        LIMIT ?
    """, [min_tours_count, limit])

    result = list()

    for row in res.fetchall():
        result.append(dict(row))
    cursor.close()

    return result


# read data
df = parse_file("tasks/task_1_var_55_item.pkl")

# connect to db
conn = connect_to_db("db2")

# inser data into table
insert_data(conn, df)

sorted_df = get_top_by_rating(conn, 55+10)
with open("results/task1_sorted_by_rating.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(sorted_df, ensure_ascii=False))

stats = get_stat_by_time(conn)
with open("results/task1_stats_by_time.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(stats, ensure_ascii=False))

freq = get_freq_by_system(conn)
with open("results/task1_freq_system.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(freq, ensure_ascii=False))

filtered_df = filter_by_tours(conn, min_tours_count=10, limit=54+10)
with open("results/task1_filter_min_tours.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(filtered_df, ensure_ascii=False))

# Вторая задача

In [ ]:
import json
import sqlite3
import msgpack


def load_data(file_name):
    with open(file_name, "rb") as f:
        byte_data = f.read()
    data = msgpack.unpackb(byte_data)
    return data


def connect_to_db(file_name):
    connection = sqlite3.connect(file_name)
    connection.row_factory = sqlite3.Row

    return connection


def insert_data(db, data):
    cursor = db.cursor()
    cursor.execute("""CREATE TABLE IF NOT EXISTS places (name text, place int, prise int)""")

    cursor.executemany("""
        INSERT INTO places (name, place, prise)
        VALUES(
            :name, :place, :prise
        )
    """, data)

    db.commit()


def first_query(db, title):
    cursor = db.cursor()
    res = cursor.execute("""
        SELECT *
        FROM places
        WHERE name = (SELECT name FROM tournaments WHERE city = ?)
        """, [title])
    items = []
    for row in res.fetchall():
        item = dict(row)
        items.append(item)

    cursor.close()
    return items


def second_query(db, title):
    cursor = db.cursor()
    res = cursor.execute("""
        SELECT
            SUM(prise) AS total_prise
        FROM places
        WHERE name = (SELECT name from tournaments WHERE city = ?)
    """, [title])
    result = dict(res.fetchone())

    cursor.close()
    return result


def third_query(db):
    cursor = db.cursor()
    res = cursor.execute("""
        SELECT t.system,
                MAX(p.prise) AS max_prise
        FROM places p
        LEFT JOIN tournaments t ON p.name = t.name
        GROUP BY t.system
    """)
    items = []
    for row in res.fetchall():
        item = dict(row)
        items.append(item)

    cursor.close()
    return items


df = load_data("tasks/task_2_var_55_subitem.msgpack")
conn = connect_to_db("db2")

# insert_data(conn, df)

first = first_query(conn, "Барселона")
with open("results/task2_query_1.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(first, ensure_ascii=False))

second = second_query(conn, "Барселона")
with open("results/task2_query_2.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(second, ensure_ascii=False))

third = third_query(conn)
with open("results/task2_query_3.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(third, ensure_ascii=False))

# Третья задача

In [ ]:
import json
import sqlite3

import pandas as pd


def load_pickle(file_name):
    data = pd.read_pickle(file_name)
    for item in data:
        del item["acousticness"]
        del item["energy"]

        item["duration_ms"] = int(item["duration_ms"])
        item["year"] = int(item["year"])
        item["tempo"] = float(item["tempo"])
        item["popularity"] = int(item["popularity"])

    return data


def load_json(file_name):
    with open(file_name, "r", encoding="utf-8") as input:
        data = json.load(input)
        for item in data:
            del item["explicit"]
            del item["danceability"]

            item["duration_ms"] = int(item["duration_ms"])
            item["year"] = int(item["year"])
            item["tempo"] = float(item["tempo"])
            item["popularity"] = int(item["popularity"])

    return data


def connect_to_db(file_name):
    connection = sqlite3.connect(file_name)
    connection.row_factory = sqlite3.Row

    return connection


def insert_data(db, data):
    cursor = db.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS songs (artist text, song text, duration_ms int, year int,
                                        tempo real, genre text, popularity int)""")

    cursor.executemany("""
        INSERT INTO songs (artist, song, duration_ms, year, tempo, genre, popularity)
        VALUES(
            :artist, :song, :duration_ms, :year, :tempo, :genre, :popularity
        )
    """, data)

    db.commit()


def get_top_by_popularity(db, limit):
    cursor = db.cursor()
    res = cursor.execute("SELECT * FROM songs ORDER BY popularity DESC LIMIT ?", [limit])
    items = list()

    for row in res.fetchall():
        items.append(dict(row))
    cursor.close()

    return items


def get_stat_by_duration(db):
    cursor = db.cursor()
    res = cursor.execute("""
        SELECT
            SUM(duration_ms) as sum,
            AVG(duration_ms) as avg,
            MIN(duration_ms) as min,
            MAX(duration_ms) as max
        FROM songs
    """)

    result = dict(res.fetchone())
    cursor.close()

    return result


def get_freq_by_artist(db):
    cursor = db.cursor()
    res = cursor.execute("""
        SELECT
            artist,
            COUNT(song) as cnt
        FROM songs
        GROUP BY artist
    """)

    result = list()

    for row in res.fetchall():
        result.append(dict(row))
    cursor.close()

    return result


def filter_by_tempo(db, min_tempo, limit=10):
    cursor = db.cursor()
    res = cursor.execute("""
        SELECT *
        FROM songs
        WHERE tempo > ?
        ORDER BY popularity DESC
        LIMIT ?
    """, [min_tempo, limit])

    result = list()

    for row in res.fetchall():
        result.append(dict(row))
    cursor.close()

    return result


songs = load_pickle('tasks/task_3_var_55_part_1.pkl') + load_json('tasks/task_3_var_55_part_2.json')

conn = connect_to_db('db3')

insert_data(conn, songs)

top_popularity = get_top_by_popularity(conn, 54+10)
with open("results/task3_top_popularity.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(top_popularity, ensure_ascii=False))

duration_stats = get_stat_by_duration(conn)
with open("results/task3_duration_stats.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(duration_stats, ensure_ascii=False))

artist_freq = get_freq_by_artist(conn)
with open("results/task3_artist_freq.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(artist_freq, ensure_ascii=False))

filtered_songs = filter_by_tempo(conn, 150, 54+15)
with open("results/task3_filtered_songs.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(filtered_songs, ensure_ascii=False))

# Четвёртая задача


In [ ]:
import json
import sqlite3
import pandas as pd


def load_data(file_name):
    data = pd.read_csv(file_name, delimiter=";") \
            .fillna({"category": "no"}) \
            .groupby("name", as_index=False) \
            .first() \
            .to_dict("records")
    return data


def load_pickle(file_name):
    data = pd.read_pickle(file_name)
    return data


def connect_to_db(file_name):
    connection = sqlite3.connect(file_name)
    connection.row_factory = sqlite3.Row

    return connection


def insert_data(db, data):
    cursor = db.cursor()
    cursor.execute("DROP TABLE IF EXISTS products")
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS products (name text UNIQUE, price real, quantity int, category text,
                                        fromCity text, isAvailable int, views int, version int DEFAULT 0)""")

    cursor.executemany("""
        INSERT INTO products (name, price, quantity, category, fromCity, isAvailable, views)
        VALUES(
            :name, :price, :quantity, :category, :fromCity, :isAvailable, :views
        )
    """, data)
    db.commit()


def delete_by_name(db, name):
    cursor = db.cursor()
    cursor.execute("DELETE FROM products WHERE name = ?", [name])
    db.commit()


def update_price_by_percent(db, name, percent):
    cursor = db.cursor()
    cursor.execute("UPDATE products SET price = ROUND((price * (1 + ?)), 2) WHERE name = ?", [percent, name])
    cursor.execute("UPDATE products SET version = version + 1 WHERE name = ?", [name])
    db.commit()


def update_price(db, name, value):
    cursor = db.cursor()
    res = cursor.execute("UPDATE products SET price = (price + ?) WHERE (name = ?) AND ((price + ?) > 0) ", [value, name, value])
    if res.rowcount > 0:
        cursor.execute("UPDATE products SET version = version + 1 WHERE name = ?", [name])
        db.commit()


def update_available(db, name, param):
    cursor = db.cursor()
    cursor.execute("UPDATE products SET isAvailable = ? WHERE (name = ?)", [param, name])
    cursor.execute("UPDATE products SET version = version + 1 where name = ?", [name])
    db.commit()


def update_quantity(db, name, value):
    cursor = db.cursor()
    res = cursor.execute("UPDATE products SET quantity = (quantity + ?) WHERE (name = ?) AND ((quantity + ?) > 0)",
                         [value, name, value])
    if res.rowcount > 0:
        cursor.execute("UPDATE products SET version = version + 1 WHERE name = ?", [name])
        db.commit()


def handle_update(db, update_items):
    for item in update_items:
        match item["method"]:
            case "remove":
                delete_by_name(db, item["name"])
            case "price_percent":
                update_price_by_percent(db, item["name"], item["param"])
            case "price_abs":
                update_price(db, item["name"], item["param"])
            case "quantity_add" | "quantity_sub":
                update_quantity(db, item["name"], item["param"])
            case "available":
                update_available(db, item["name"], item["param"])
            case _:
                print(f"unknown method {item['method']}")


def get_top_updated(db, limit=10):
    cursor = db.cursor()
    res = cursor.execute("SELECT name, version FROM products ORDER BY version DESC LIMIT ?", [limit])
    items = list()

    for row in res.fetchall():
        items.append(dict(row))
    cursor.close()

    return items


def get_group_price_stat(db):
    cursor = db.cursor()
    res = cursor.execute("""
    SELECT category,
           SUM(price) AS sum_price,
           MIN(price) AS min_price,
           MAX(price) AS max_price,
           AVG(price) AS avg_price,
           COUNT(name) AS products_count
    FROM products
    GROUP BY category""")
    items = list()

    for row in res.fetchall():
        items.append(dict(row))
    cursor.close()

    return items


def get_group_quantity_stat(db):
    cursor = db.cursor()
    res = cursor.execute("""
    SELECT category,
           SUM(quantity) AS sum_quantity,
           MIN(quantity) AS min_quantity,
           MAX(quantity) AS max_quantity,
           AVG(quantity) AS avg_quantity
    FROM products
    GROUP BY category""")
    items = list()

    for row in res.fetchall():
        items.append(dict(row))
    cursor.close()

    return items


def get_top_viewed_products(db, limit=10):
    cursor = db.cursor()
    res = cursor.execute("""
    SELECT name,
           views
    FROM products
    GROUP BY name
    ORDER BY views DESC
    LIMIT ?""", [limit])
    items = list()

    for row in res.fetchall():
        items.append(dict(row))
    cursor.close()

    return items


products = load_data("tasks/task_4_var_55_product_data.csv")
upd = load_pickle("tasks/task_4_var_54_update_data.pkl")
conn = connect_to_db("db4")

# insert_data(conn, products)
# handle_update(conn, upd)
top_updated = get_top_updated(conn, 10)
with open("results/task4_top_updated.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(top_updated, ensure_ascii=False))

category_price_stats = get_group_price_stat(conn)
with open("results/task4_category_price_stats.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(category_price_stats, ensure_ascii=False))

category_quantity_stats = get_group_quantity_stat(conn)
with open("results/task4_category_quantity_stats.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(category_quantity_stats, ensure_ascii=False))

top_viewed = get_top_viewed_products(conn, 10)
with open("results/task4_top_viewed.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(top_viewed, ensure_ascii=False))

# Пятая задача

In [ ]:
import sqlite3
import csv
import json


def connect_to_db(db_name):
    connection = sqlite3.connect(db_name)

    return connection


# Создание таблиц
def create_table(connection):
    cursor = connection.cursor()
    cursor.executescript(
        """
        DROP TABLE IF EXISTS games;
        DROP TABLE IF EXISTS users;
        DROP TABLE IF EXISTS recommendations;
        DROP TABLE IF EXISTS games_metadata;

        CREATE TABLE games(
            app_id int primary key,
            title text,
            date_release text,
            win text,
            mac text,
            linux text,
            rating text,
            positive_ratio int,
            user_reviews int,
            price_final real,
            price_original real,
            discount real,
            steam_deck text
        );

        CREATE TABLE users(
            user_id int primary key,
            products int,
            reviews int
        );

        CREATE TABLE recommendations(
            app_id int,
            helpful int,
            funny int,
            date text,
            is_recommended text,
            hours real,
            user_id int,
            review_id int primary key,
            FOREIGN KEY(app_id) REFERENCES games(app_id),
            FOREIGN KEY(user_id) REFERENCES users(user_id)
        );

        CREATE TABLE games_metadata(
            app_id int,
            description text,
            tags text,
            FOREIGN KEY(app_id) REFERENCES games(app_id)
        );
        """)

    connection.commit()


# Наполнение таблиц
def insert_into_games_csv(connection, file_name):
    cursor = connection.cursor()

    with open(file_name, 'r', encoding='utf-8') as f:
        next(f)
        rows = csv.reader(f, delimiter=",")

        for row in rows:
            cursor.execute('INSERT INTO games VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', row)

        connection.commit()


def insert_into_users_csv(connection, file_name):
    cursor = connection.cursor()

    with open(file_name, 'r', encoding='utf-8') as f:
        next(f)
        rows = csv.reader(f, delimiter=",")

        for row in rows:
            cursor.execute('INSERT INTO users VALUES (?, ?, ?)', row)

        connection.commit()


def insert_into_recommendations_csv(connection, file_name):
    cursor = connection.cursor()

    with open(file_name, 'r', encoding='utf-8') as f:
        next(f)
        rows = csv.reader(f, delimiter=",")

        for row in rows:
            cursor.execute('INSERT INTO recommendations VALUES (?, ?, ?, ?, ?, ?, ?, ?)', row)

        connection.commit()


def insert_into_games_metadata_json(connection, file_name):
    cursor = connection.cursor()

    rows = [json.loads(line) for line in open(file_name, 'r', encoding='utf-8')]

    for row in rows:
        cursor.execute('INSERT INTO games_metadata VALUES (?, ?, ?)',
                       (row['app_id'], row['description'], ', '.join(map(str, row['tags']))))
    connection.commit()


# Запросы к бд
def get_count_per_year(connection):
    """
    Нахождение топ 10 лет в 21 веке с самым большим количеством игр за год с позитивной оценкой
    """
    cursor = connection.cursor()
    res = cursor.execute(
        """
        SELECT strftime('%Y', date_release), COUNT(app_id) as cnt
        FROM games
        WHERE LOWER(rating) LIKE '%positive%'
        GROUP BY strftime('%Y', date_release)
        ORDER BY COUNT(app_id) DESC
        LIMIT 10;
        """)

    result = dict(res.fetchall())
    cursor.close()

    return result


def get_games_on_all_os(connection):
    """
    Нахождение ирг, которые поддерживаются и на macOS, и Linux, и Windows
    """
    cursor = connection.cursor()
    res = cursor.execute(
        """
        SELECT app_id, title
        FROM games
        WHERE mac = 'true'
            AND linux = 'true'
            AND win = 'true'
        """)

    result = dict(res.fetchall())
    cursor.close()

    return result


def get_max_reviews_products(connection):
    """
    Нахождение пользователя с наибольшим количеством игр и пользователя с наибольшим количеством отзывов
    """
    cursor = connection.cursor()
    res = cursor.execute(
        """
        SELECT *
        FROM users
        WHERE reviews = (SELECT MAX(reviews) FROM users)
            OR products = (SELECT MAX(products) FROM users)
        """)

    result = {}
    for a, b, c in res:
        result[a] = (a, b, c)
    cursor.close()

    return result


def get_max_hours(connection):
    """
    Нахождение игры, в которой оставивший отзыв пользователь провел больше всего часов
    """
    cursor = connection.cursor()
    res = cursor.execute(
        """
        SELECT r.hours, g.title
        FROM users u
        JOIN recommendations r ON u.user_id = r.user_id
        JOIN games g ON g.app_id = r.app_id
        WHERE r.hours in (SELECT MAX(hours) FROM recommendations)
        """)

    result = dict(res.fetchall())
    cursor.close()

    return result


def get_most_rec_game(connection):
    """
    Нахождение игры, которую рекомендует больше всего игроков
    """
    cursor = connection.cursor()
    res = cursor.execute(
        """
        SELECT g.title, COUNT(r.review_id) as cnt
        FROM recommendations r
        JOIN games g ON g.app_id = r.app_id
        WHERE r.is_recommended = 'true'
        GROUP BY g.title
        ORDER BY COUNT(r.review_id) DESC
        LIMIT 1
        """)

    result = dict(res.fetchall())
    cursor.close()

    return result


def delete_last_10_years(connection):
    """
    Удаление рекомендаций созданных более 10 лет назад
    """
    cursor = connection.cursor()
    cursor.execute(
        """
        DELETE FROM recommendations
        WHERE CAST(strftime('%Y', date) AS INTEGER) <= CAST(date('now', '-10 years') AS INTEGER)
        """)


conn = connect_to_db('db5')
create_table(conn)
insert_into_games_csv(conn, 'data/games.csv')
insert_into_users_csv(conn, 'data/users.csv')
insert_into_recommendations_csv(conn, 'data/recommendations.csv')
insert_into_games_metadata_json(conn, 'data/games_metadata.json')

with open('results/task5_get_count_per_year.json', "w", encoding="utf-8") as f:
    f.write(json.dumps(get_count_per_year(conn), ensure_ascii=False))

with open('results/task5_get_games_on_all_os.json', "w", encoding="utf-8") as f:
    f.write(json.dumps(get_games_on_all_os(conn), ensure_ascii=False))

with open('results/task5_get_max_reviews_products.json', "w", encoding="utf-8") as f:
    f.write(json.dumps(get_max_reviews_products(conn), ensure_ascii=False))

with open('results/task5_get_max_hours.json', "w", encoding="utf-8") as f:
    f.write(json.dumps(get_max_hours(conn), ensure_ascii=False))

with open('results/task5_get_most_rec_game.json', "w", encoding="utf-8") as f:
    f.write(json.dumps(get_most_rec_game(conn), ensure_ascii=False))

delete_last_10_years(conn)